# **Imports librairies et données**

In [ ]:
import pandas as pd
import math
import scipy.sparse as sps
from scipy.sparse import csr_matrix
from sklearn.preprocessing import MinMaxScaler

In [ ]:
train = pd.read_csv("/content/drive/MyDrive/Challenges/bnp_dataset/X_train.csv")
test = pd.read_csv("/content/drive/MyDrive/Challenges/bnp_dataset/X_test.csv")
target = pd.read_csv("/content/drive/MyDrive/Challenges/bnp_dataset/Y_train.csv")

<ipython-input-2-341ab314565f>:1: DtypeWarning: Columns (21,22,23,24,69,70,71,72,93,94,95,96,97,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv("/content/drive/MyDrive/Challenges/bnp_dataset/X_train.csv")
<ipython-input-2-341ab314565f>:2: DtypeWarning: Columns (20,21,22,23,24,68,69,70,71,72,92,93,94,95,96,97,106,107,108,109,110,111,112,113,114,115,116,117,118) have mixed types. Specify dtype option on import or set low_memory=False.
  test = pd.read_csv("/content/drive/MyDrive/Challenges/bnp_dataset/X_test.csv")


In [ ]:
index_fraud = target[target["fraud_flag"]==1].index

# **Nettoyage du texte**

In [ ]:
def clean(x):
  if type(x)==str:
    x = x.replace("& ","")
    x = x.replace(", ","")
    x = x.replace(" S ","S ")
    x = x.replace('TELEPHONESFAX','TELEPHONES FAX')
    x = x.replace('CARPETSRUGS','CARPETS RUGS')
    x = x.replace('BAGSWALLETS','BAGS WALLETS')
    x = x.replace('!','')
    x = x.replace('BABYBJ RN','BABYBJÖRN')
    x = x.replace('DR BROWN S','DR BROWNS')
    x = x.replace('KIEHL S','KIEHLS')
    x = x.replace('®','')
    x = x.replace('.','')
    x = x.replace('2','')
    x = x.replace(" + ","")
  return x

# **Création des matrices**

## **Item**

In [ ]:
items = []
for i in range(1,25):
  items += train[f"item{i}"].unique().tolist()
  #items += test[f"item{i}"].unique().tolist()
items = set(items)

In [ ]:
items = list(items)

In [ ]:
items[0]

nan

In [ ]:
del items[0]

In [ ]:
train_item_price = pd.DataFrame(columns=items,index=train.index).fillna(0)
train_item_count = pd.DataFrame(columns=items,index=train.index).fillna(0)

In [ ]:
for row in train.index:
  for i in range(1,25):
    item = train[f"item{i}"][row]
    if item in items:
     train_item_price[item][row] += train[f"cash_price{i}"][row] * train[f"Nbr_of_prod_purchas{i}"][row]
     train_item_count[item][row] += train[f"Nbr_of_prod_purchas{i}"][row]

In [ ]:
train_item_price

,TELEVISIONS HOME CINEMA,CHILDREN S FURNITURE,MEN S NIGHTWEAR,TARGUS GEOLITE ESSENTIAL CASE,HOUSEHOLD CLEANING,POWER & BATTERIES,SCHOOLWEAR,COMPUTER SOFTWARE,UNKNOWN,FILING DESK ACCESSORIES,...,STANDS & BRACKETS,MEN S SPORTSWEAR,HOME SAFETY EQUIPMENT,TOYS,SUNGLASSES READING GLASSES,2LOGITECH PEBBLE M350 BLUETOOTH MOUSE,PRESERVING & BAKING EQUIPMENT,LUGGAGE,GIFT FOOD DRINK,WOMENS FOOTWEAR
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1399,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1199,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92785,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
92786,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
92787,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
92788,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
train_item_price.to_csv(f'drive/MyDrive/Challenges/bnp_dataset/final/train_item_price_scale.csv',index=False)
train_item_count.to_csv(f'drive/MyDrive/Challenges/bnp_dataset/final/train_item_count.csv',index=False)

# **Make**

In [ ]:
makes = []
for i in range(1,25):
  makes += train[f"make{i}"].unique().tolist()
  #makes += test[f"make{i}"].unique().tolist()
makes = set(makes)

In [ ]:
makes = list(makes)

In [ ]:
makes[0]

nan

In [ ]:
del makes[0]

In [ ]:
train_make_price = pd.DataFrame(columns=makes,index=train.index).fillna(0)
train_make_count = pd.DataFrame(columns=makes,index=train.index).fillna(0)

In [ ]:
for row in train.index:
  for i in range(1,25):
    make = train[f"make{i}"][row]
    if make in makes:
     train_make_price[make][row] += train[f"cash_price{i}"][row] * train[f"Nbr_of_prod_purchas{i}"][row]
     train_make_count[make][row] += train[f"Nbr_of_prod_purchas{i}"][row]

In [ ]:
train_make_price.to_csv(f'drive/MyDrive/Challenges/bnp_dataset/final/train_make_price_scale.csv',index=False)
train_make_count.to_csv(f'drive/MyDrive/Challenges/bnp_dataset/final/train_make_count.csv',index=False)

## **Model**

In [ ]:
models = []
for i in range(1,25):
  models += train[f"model{i}"].unique().tolist()
  #models += test[f"model{i}"].unique().tolist()
models = set(models)

In [ ]:
models = list(models)
models[0]

nan

In [ ]:
del models[0]

In [ ]:
len(models)

9679

In [ ]:
train_model_count = pd.DataFrame(columns=models,index=train.index).fillna(0)

In [ ]:
train_model_count

,FRENCH CONNECTION BOZA RUFFLE NECK BLOUSE DEEP MOS,RETAILER BOYS MID WASH DENIM SHORTS MID,D R SOREN TEXTURED GLASS CEILING LIGHT SMOKE,THE HOME EDIT IDESIGN BERRY FRIDGE BIN SMALL,SILENTNIGHT SLEEP GENIUS 2200 POCKET LATEX MATTRES,RETAILER SANTINO DINING CHAIR TAN OAK,BRITA MAXTRA ELEMARIS XL LARGE WATER FILTER JUG 3,RETAILER GEORGEY LEATHER CHUNKY FLATFOR,RUCOMFY INDOOR OUTDOOR BEAN CUBE NAVY,APPLE 61W USB-C POWER ADAPTER,...,APPLE IPHONE 12 PRO,SONY BRAVIA XR XR85X95J 2021 LED HDR 4K ULTRA HD S,RETAILER WATER HYACINTH WASTE PAPER BIN,MORRIS CO PIMPERNEL RUG L170 X W240 CM BULLRUSH,OSCO BAMBOO STACKABLE FILE,RETAILER PANEL SMALL 2 SEATER SOFA BLAC,TOMMEE TIPPEE MADE FOR ME SINGLE SILICONE BREAST P,IH201840,MJYT2ZM/A,RETAILER VELVET BLACKOUT ROMAN BLIND GR
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92785,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
92786,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
92787,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
92788,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
for row in train.index:
  for i in range(1,25):
    model = train[f"model{i}"][row]
    if model in models:
      train_model_count[model][row] += train[f"Nbr_of_prod_purchas{i}"][row]

In [ ]:
train_model_count.to_csv(f'drive/MyDrive/Challenges/bnp_dataset/final/train_model_count_fraud.csv',index=False)

In [ ]:
train_item_price = pd.DataFrame(columns=items)
train_item_count = pd.DataFrame(columns=items)
train_make_price = pd.DataFrame(columns=makes)
train_make_count = pd.DataFrame(columns=makes)
train_model_count = pd.DataFrame(columns=models)
train_item_price.to_csv(f'drive/MyDrive/Challenges/bnp_dataset/final/train_item_price_scale.csv',index=False)
train_item_count.to_csv(f'drive/MyDrive/Challenges/bnp_dataset/final/train_item_count.csv',index=False)
train_make_price.to_csv(f'drive/MyDrive/Challenges/bnp_dataset/final/train_make_price_scale.csv',index=False)
train_make_count.to_csv(f'drive/MyDrive/Challenges/bnp_dataset/final/train_make_count.csv',index=False)
train_model_count.to_csv(f'drive/MyDrive/Challenges/bnp_dataset/final/train_model_count_fraud.csv',index=False)

In [ ]:
train_model_count = pd.DataFrame(columns=models)
train_model_count.to_csv(f'drive/MyDrive/Challenges/bnp_dataset/final/train_model_count_fraud.csv',index=False)